__Assignment 3__

3) Find the average monthly temperature for each available station in Sweden. Your result should include average temperature for each station for each month in the period of 1960- 2014. Bear in mind that not every station has the readings for each month in this timeframe. In this exercise you will use the *temperature-readings.csv file*.

The output should contain the following information:
__Year, month, station number, average monthly temperature__

For each question include the following data in the report and sort it as shown:
- year, month, station, avgMonthlyTemperature ORDER BY avgMonthlyTemperature DESC

In [1]:
# create the setup
from pyspark import SparkContext
from pyspark.sql import SQLContext, Row  
from pyspark.sql import functions as F
sc =SparkContext()
sqlContext=SQLContext(sc)

In [29]:
from pyspark.sql.functions import col, round, upper

In [2]:
path_unifolder_temp_tiny = "/Users/phillipholscher/OneDrive - Linköpings universitet/LiU/Statistics and Machine Learning/Spring19T2/Big Data Analytics/Labs/Lab 2/temperature-readings-tiny.csv"


In [3]:
# read and create rdd, this needs to be tranfsormed to data frame later on
temps= sc.textFile(path_unifolder_temp_tiny)
parts = temps.map(lambda l:l.split(";"))
tempReadings = parts.map(lambda p: Row(station=p[0],  date=p[1], year=p[1].split("-")[0], time=p[2],  Temp=float(p[3]), quality=p[4], month=p[1].split("-")[1]))
tempReadings.take(10)


[Row(Temp=6.8, date='2013-11-01', month='11', quality='G', station='102170', time='06:00:00', year='2013'),
 Row(Temp=3.8, date='2013-11-01', month='11', quality='G', station='102170', time='18:00:00', year='2013'),
 Row(Temp=5.8, date='2013-11-02', month='11', quality='G', station='102170', time='06:00:00', year='2013'),
 Row(Temp=-1.1, date='2013-11-02', month='11', quality='G', station='102170', time='18:00:00', year='2013'),
 Row(Temp=-0.2, date='2013-11-03', month='11', quality='G', station='102170', time='06:00:00', year='2013'),
 Row(Temp=5.6, date='2013-11-03', month='11', quality='G', station='102170', time='18:00:00', year='2013'),
 Row(Temp=6.5, date='2013-11-04', month='11', quality='G', station='102170', time='06:00:00', year='2013'),
 Row(Temp=5.1, date='2013-11-04', month='11', quality='G', station='102170', time='18:00:00', year='2013'),
 Row(Temp=4.2, date='2013-11-05', month='11', quality='G', station='102170', time='06:00:00', year='2013'),
 Row(Temp=3.2, date='2013-

In [4]:
# create a data frame from rdd
schemaTempReadings =  sqlContext.createDataFrame(tempReadings)
schemaTempReadings.registerTempTable("tempReadings")
schemaTempReadings.show(10)


+----+----------+-----+-------+-------+--------+----+
|Temp|      date|month|quality|station|    time|year|
+----+----------+-----+-------+-------+--------+----+
| 6.8|2013-11-01|   11|      G| 102170|06:00:00|2013|
| 3.8|2013-11-01|   11|      G| 102170|18:00:00|2013|
| 5.8|2013-11-02|   11|      G| 102170|06:00:00|2013|
|-1.1|2013-11-02|   11|      G| 102170|18:00:00|2013|
|-0.2|2013-11-03|   11|      G| 102170|06:00:00|2013|
| 5.6|2013-11-03|   11|      G| 102170|18:00:00|2013|
| 6.5|2013-11-04|   11|      G| 102170|06:00:00|2013|
| 5.1|2013-11-04|   11|      G| 102170|18:00:00|2013|
| 4.2|2013-11-05|   11|      G| 102170|06:00:00|2013|
| 3.2|2013-11-05|   11|      G| 102170|18:00:00|2013|
+----+----------+-----+-------+-------+--------+----+
only showing top 10 rows



In [8]:
# filter time interval restrictions
schemaTempReadings_filterd = schemaTempReadings.select(["year", "month","station","Temp"])\
.filter((schemaTempReadings["year"] >= 1960) & (schemaTempReadings["year"] <= 2014) )
schemaTempReadings_filterd.show(10)


+----+-----+-------+----+
|year|month|station|Temp|
+----+-----+-------+----+
|2013|   11| 102170| 6.8|
|2013|   11| 102170| 3.8|
|2013|   11| 102170| 5.8|
|2013|   11| 102170|-1.1|
|2013|   11| 102170|-0.2|
|2013|   11| 102170| 5.6|
|2013|   11| 102170| 6.5|
|2013|   11| 102170| 5.1|
|2013|   11| 102170| 4.2|
|2013|   11| 102170| 3.2|
+----+-----+-------+----+
only showing top 10 rows



In [14]:
# to calculate the monthly avg for each station do we need the max and min temp for each month and station
schemaTempReadings_max_min = schemaTempReadings_filterd.groupBy("year", "month", "station")\
.agg(F.max("Temp").alias("Temp_max"),F.min("Temp").alias("Temp_min"))
schemaTempReadings_max_min.show(10)


+----+-----+-------+--------+--------+
|year|month|station|Temp_max|Temp_min|
+----+-----+-------+--------+--------+
|2014|   01| 102170|     6.6|   -19.6|
|1960|   07| 102190|    25.0|     6.0|
|2014|   10| 102170|    14.7|    -4.5|
|1960|   08| 102190|    28.0|     2.8|
|1961|   01| 102190|     3.1|   -23.5|
|2014|   12| 102170|     5.6|   -24.3|
|2014|   09| 102170|    17.6|    -2.3|
|1960|   01| 102190|     2.9|   -20.4|
|2014|   05| 102170|    24.8|     0.3|
|1960|   06| 102190|    29.0|     5.2|
+----+-----+-------+--------+--------+
only showing top 10 rows



In [16]:
def avg_temp(max_temp, min_temp):
    avg = (max_temp + min_temp)/2
    return avg

In [34]:
# compute the avg monthly temp for each station
schemaTempReadings_avg = schemaTempReadings_max_min.withColumn("Temp_avg", 
                                                               round(avg_temp(schemaTempReadings_max_min["Temp_max"],
                                                                       schemaTempReadings_max_min["Temp_min"]),1))
schemaTempReadings_avg.show(10)


+----+-----+-------+--------+--------+--------+
|year|month|station|Temp_max|Temp_min|Temp_avg|
+----+-----+-------+--------+--------+--------+
|2014|   01| 102170|     6.6|   -19.6|    -6.5|
|1960|   07| 102190|    25.0|     6.0|    15.5|
|2014|   10| 102170|    14.7|    -4.5|     5.1|
|1960|   08| 102190|    28.0|     2.8|    15.4|
|1961|   01| 102190|     3.1|   -23.5|   -10.2|
|2014|   12| 102170|     5.6|   -24.3|    -9.4|
|2014|   09| 102170|    17.6|    -2.3|     7.7|
|1960|   01| 102190|     2.9|   -20.4|    -8.8|
|2014|   05| 102170|    24.8|     0.3|    12.6|
|1960|   06| 102190|    29.0|     5.2|    17.1|
+----+-----+-------+--------+--------+--------+
only showing top 10 rows



In [37]:
# sort the avg temp 
schemaTempReadings_avg_ordered = schemaTempReadings_avg.select(["year", "month", "station", "Temp_avg"])\
.orderBy("Temp_avg", ascending = False)

schemaTempReadings_avg_ordered.show(10)


+----+-----+-------+--------+
|year|month|station|Temp_avg|
+----+-----+-------+--------+
|2014|   07| 102170|    20.4|
|1960|   06| 102190|    17.1|
|1960|   07| 102190|    15.5|
|1960|   08| 102190|    15.4|
|2014|   06| 102170|    15.2|
|2014|   08| 102170|    14.8|
|2014|   05| 102170|    12.6|
|1961|   05| 102190|    10.6|
|1960|   05| 102190|     9.9|
|1960|   09| 102190|     9.3|
+----+-----+-------+--------+
only showing top 10 rows

